In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array, save_img, array_to_img
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


# 1. Load dataset
# ------------- Provide your dataset paths here ----------------
IMAGE_PATH = "C:\Dataset 1\Batch"
MASK_PATH = "C:\Mask 1\Batch"
# --------------------------------------------------------------

image_files = sorted([os.path.join(IMAGE_PATH, f) for f in os.listdir(IMAGE_PATH)])
mask_files = sorted([os.path.join(MASK_PATH, f) for f in os.listdir(MASK_PATH)])

images = [img_to_array(load_img(img)) / 255. for img in image_files]
masks = [img_to_array(load_img(m, color_mode='grayscale')) / 255. for m in mask_files]

# Split data into train and test sets
train_images, test_images, train_masks, test_masks = train_test_split(images, masks, test_size=0.2)


In [ ]:
def build_fcn(input_shape):
    inputs = Input(shape=input_shape)
    
    # Encoder
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    # Decoder
    x = Conv2DTranspose(64, (2, 2), strides=(2, 2), activation='relu')(x)
    x = Conv2DTranspose(32, (2, 2), strides=(2, 2), activation='relu')(x)
    
    # Output layer
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

model = build_fcn(train_images[0].shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(np.array(train_images), np.array(train_masks), validation_data=(np.array(test_images), np.array(test_masks)), epochs=10, batch_size=32)

In [ ]:
# 4. Predict on test images
predictions = model.predict(np.array(test_images))

# 5. Save output segmentation masks
OUTPUT_FOLDER = "C:\Deep learning\FCN output"
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

# Combine train and test images
all_images = np.concatenate((train_images, test_images), axis=0)

# Predict on the entire dataset
all_predictions = model.predict(all_images)

# Save the predicted masks to the designated folder
for idx, pred in enumerate(all_predictions):
    # Convert the prediction to binary format (optional, based on your needs)
    pred_binary = (pred > 0.5).astype(np.uint8)
    
    # Remove singleton dimensions
    pred_squeezed = np.squeeze(pred_binary)

    # Convert the 2D mask to a 3D tensor
    pred_3d = np.expand_dims(pred_squeezed, axis=-1)

    # Save the mask
    mask_save_path = os.path.join(OUTPUT_FOLDER, f"predicted_mask_{idx}.png")
    save_img(mask_save_path, pred_3d * 255)  # Multiply by 255 to convert from [0, 1] to [0, 255] range

print(f"Saved {len(all_predictions)} predicted masks to {OUTPUT_FOLDER}")

In [ ]:
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

# Combine train and test images
all_images = np.concatenate((train_images, test_images), axis=0)

# Predict on the entire dataset
all_predictions = model.predict(all_images)

# Save the predicted masks to the designated folder
for idx, pred in enumerate(all_predictions):
    # Convert the prediction to binary format (optional, based on your needs)
    pred_binary = (pred > 0.5).astype(np.uint8)
    
    # Remove singleton dimensions
    pred_squeezed = np.squeeze(pred_binary)

    # Convert the 2D mask to a 3D tensor
    pred_3d = np.expand_dims(pred_squeezed, axis=-1)

    # Save the mask
    mask_save_path = os.path.join(OUTPUT_FOLDER, f"predicted_mask_{idx}.png")
    save_img(mask_save_path, pred_3d * 255)  # Multiply by 255 to convert from [0, 1] to [0, 255] range

print(f"Saved {len(all_predictions)} predicted masks to {OUTPUT_FOLDER}")

In [ ]:
def visualize_samples(images, masks, num_samples=15):
    fig, axs = plt.subplots(num_samples, 2, figsize=(10, 5*num_samples))
    
    for i in range(num_samples):
        axs[i, 0].imshow(images[i])
        axs[i, 0].set_title('Image')
        
        axs[i, 1].imshow(masks[i].squeeze(), cmap='gray')
        axs[i, 1].set_title('Mask')
        
        axs[i, 0].axis('off')
        axs[i, 1].axis('off')

    plt.tight_layout()
    plt.show()

visualize_samples(test_images, test_masks)

# ... [rest of the code]

# 2. Visualize predictions
def visualize_predictions(originals, true_masks, predictions, num_samples=3):
    fig, axs = plt.subplots(num_samples, 3, figsize=(15, 5*num_samples))
    
    for i in range(num_samples):
        axs[i, 0].imshow(originals[i])
        axs[i, 0].set_title('Original Image')
        
        axs[i, 1].imshow(true_masks[i].squeeze(), cmap='gray')
        axs[i, 1].set_title('True Mask')
        
        axs[i, 2].imshow(predictions[i].squeeze(), cmap='gray')
        axs[i, 2].set_title('Predicted Mask')
        
        axs[i, 0].axis('off')
        axs[i, 1].axis('off')
        axs[i, 2].axis('off')

    plt.tight_layout()
    plt.show()

visualize_predictions(test_images, test_masks, predictions)

In [ ]:
import matplotlib.pyplot as plt

def apply_color_map(image, mask):
    """Apply the specified color map to the given image based on the mask."""
    colored_image = image.copy()
    
    # For this example, we will color the segmented parts in blue. 
    # This means the red and green channels will be zeroed out wherever the mask is 1.
    colored_image[mask.squeeze() > 0.5, 0] = 0  # Zero out the red channel
    colored_image[mask.squeeze() > 0.5, 1] = 0  # Zero out the green channel
    
    # You can adjust the RGB values to get your desired segmentation color.
    
    return colored_image

def visualize_predictions_colored(originals, predictions, num_samples=10):
    if num_samples > len(originals):
        num_samples = len(originals)

    fig, axs = plt.subplots(num_samples, 2, figsize=(10, 5*num_samples))
    
    for i in range(num_samples):
        colored_output = apply_color_map(originals[i], predictions[i])
        
        axs[i, 0].imshow(originals[i])
        axs[i, 0].set_title('Original Image')
        
        axs[i, 1].imshow(colored_output)
        axs[i, 1].set_title('Segmentation Overlay')
        
        axs[i, 0].axis('off')
        axs[i, 1].axis('off')

    plt.tight_layout()
    plt.show()

# Assuming test_images contain your images and predictions contain your predicted masks
visualize_predictions_colored(test_images, predictions)

In [ ]:

true_mask_files = [f for f in os.listdir(MASK_PATH) if os.path.isfile(os.path.join(MASK_PATH, f)) and f.lower().endswith(('png', 'jpg', 'jpeg'))]
true_masks = [img_to_array(load_img(os.path.join(MASK_PATH, f), color_mode='grayscale')) / 255. for f in true_mask_files]
def apply_color_map(image, mask):
    """Apply the specified color map to the given image based on the mask."""
    colored_image = image.copy()
    
    # For this example, we will color the segmented parts in blue. 
    # This means the red and green channels will be zeroed out wherever the mask is 1.
    colored_image[mask.squeeze() > 0.5, 0] = 0  # Zero out the red channel
    colored_image[mask.squeeze() > 0.5, 1] = 0  # Zero out the green channel
    
    # You can adjust the RGB values to get your desired segmentation color.
    
    return colored_image

def visualize_predictions_and_true_masks(originals, predictions, true_masks, num_samples=10):
    if num_samples > len(originals):
        num_samples = len(originals)

    fig, axs = plt.subplots(num_samples, 3, figsize=(15, 5*num_samples))
    
    for i in range(num_samples):
        colored_output = apply_color_map(originals[i], predictions[i])
        true_colored_output = apply_color_map(originals[i], true_masks[i])
        
        axs[i, 0].imshow(originals[i])
        axs[i, 0].set_title('Original Image')
        
        axs[i, 1].imshow(colored_output)
        axs[i, 1].set_title('Predicted Segmentation Overlay')
        
        axs[i, 2].imshow(true_colored_output)
        axs[i, 2].set_title('True Segmentation Overlay')
        
        axs[i, 0].axis('off')
        axs[i, 1].axis('off')
        axs[i, 2].axis('off')

    plt.tight_layout()
    plt.show()

# Assuming:
# test_images contain your images
# predictions contain your predicted masks
# true_masks contain the ground truth masks for the test images
visualize_predictions_and_true_masks(test_images, predictions, true_masks)